![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and LangChain to make a series of calls to a language model

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.
- กรุณาใช้ Projects และข้อมูล ที่เตรียมไว้ให้ใน watsonx

## Notebook content

This notebook contains the steps and code to demonstrate Simple Sequential Chain using langchain integration with watsonx models.

แบบฝึกหัดนี้ประกอบไปด้วย ตัวอย่างโค้ดและขั้นตอน การทำ Simple Sequential Chain โดยใช้ langchain เชื่อต่อกับ โมเดลจาก watsonx

Some familiarity with Python is helpful. This notebook uses Python 3.10.

แบบฝึกหัดนี้ใช้ Python 3.10 เหมาะสำหรับผู้มีพื้นฐาน Python

## Learning goal

The goal of this notebook is to demonstrate how to chain `google/flan-ul2` and `google/flan-t5-xxl` models to generate a sequence of creating a random question on a given topic and an answer to that question and also to make the user friends with LangChain framework, using simple chain (LLMChain) and the extended chain (SimpleSequentialChain) with the WatsonxLLM.

เป้าหมายของแบบฝึกหัดนี้คือการสอนวิธีการ เชื่อมต่อ โมเดล 2 ส่วน ได้แก่ `google/flan-ul2` และ `google/flan-t5-xxl` เพื่อทำการสร้างการสนทนาถามและตอบ แบบง่าย โดยใช้ เฟรมเวิร์คที่เรียกว่า LangChain โดยใช้พื้นฐาน ของ simple chain (LLMChain) และขยายต่อไปสู่ chain การสนทนาที่ต่อเนื่อง (SimpleSequentialChain) โดยใช้ WatsonxLLM.


## Contents

This notebook contains the following parts:

แบบฝึกหัดนี้ประกอบไปด้วย:

- [Setup](#setup)
- [Foundation Models on watsonx](#models)
- [LangChain integration](#watsonxllm)
- [Simple Sequential Chain experiment](#experiment)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

ก่อนที่ทุกท่านจะเริ่มทำแบบฝึกหัดนี้ กรุณาทำการติดตั้ง ตามวิธีการนี้ก่อน:


-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">here</a>).


-  สร้าง <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (สามารถใช้แบบไม่มีค่าใช้จ่าย และรายละเอียด การติดตั้งเป็นดังนี้ <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">ข้อมูลเพิ่มเติม</a>).


### Install and import the `datasets` and dependecies

In [ ]:
!pip install "ibm-watson-machine-learning>=1.0.327" | tail -n 1
!pip install "pydantic>=1.10.0" | tail -n 1
!pip install langchain | tail -n 1

### Defining the WML credentials
This cell defines the WML credentials required to work with watsonx Foundation Model inferencing.

ขั้นตอนต่อไปนี้จะเป็นการระบุ credentials ที่จำเป็นต้องใช้ในการ ใช้งาน watsonx Foundation Model

**Action:** Provide the IBM Cloud user API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

**Action:** กรอกข้อมูล IBM cloud user API key รายละเอียดและวิธีการอยู่ข้างล่าางนี้
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

### Defining the project id
The Foundation Model requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.


กำหนด project id เพื่อให้สามารถเชื่อมต่อแบบฝึกหนัดนี้ไปหา watsonx ได้อย่างถูกต้อง

In [ ]:
import os
import json
from dotenv import load_dotenv
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

In [ ]:
#config Watsonx.ai environment
load_dotenv()
project_id = os.environ["PROJECT_ID"]
ibm_cloud_url = os.environ["IBM_CLOUD_URL"]
api_key = os.environ["API_KEY"]


In [ ]:
#config Watsonx.ai environment

if api_key is None or ibm_cloud_url is None or project_id is None:
    raise Exception("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
else:
    credentials = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }

<a id="models"></a>
## Foundation Models on `watsonx.ai`

#### List available models

All avaliable models are presented under ModelTypes class.
For more information refer to [documentation](https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html#ibm_watson_machine_learning.foundation_models.utils.enums.ModelTypes).


Model ประเภท LLM ทั้งหมด สามารถดูได้จาก ModelTypes
สามารถค้นหาข้อมูลเพิ่มเติมได้จาก [documentation](https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html#ibm_watson_machine_learning.foundation_models.utils.enums.ModelTypes).


In [ ]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

You need to specify `model_id` that will be used for inferencing:

ในการให้ LLM ทำงานนั้น เราเลือก `model_id` ที่เราต้องการจะใช้มัน:

In [ ]:
model_id_1 = ModelTypes.FLAN_UL2
model_id_2 = ModelTypes.FLAN_T5_XXL
model_id_3 = ModelTypes.LLAMA_2_70B_CHAT
model_id_4 = ModelTypes.GRANITE_13B_INSTRUCT
model_id_5 = ModelTypes.GPT_NEOX
model_id_6 = ModelTypes.MPT_7B_INSTRUCT2

### Defining the model parameters

You might need to adjust model `parameters` for different models or tasks, to do so please refer to documentation under `GenTextParamsMetaNames` class.

**Action:** If any complications please refer to the [documentation](https://ibm.github.io/watson-machine-learning-sdk/).



### ตั้งค่า parameters ของ model

เราอาจต้องการ การปรับ `parameters` สำหรับ โมเดลที่ต่างกัน เพื่อทำประเภทของงานที่ต่างกัน สามารถอ่านข้อมูลเพิ่มเติมได้ที่ `GenTextParamsMetaNames` class
[documentation](https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html#metanames.GenTextParamsMetaNames).

In [ ]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TEMPERATURE: 0.5,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1
}

In [ ]:
LLAMA_parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.MIN_NEW_TOKENS: 2,
    GenParams.TEMPERATURE: 0.5,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1
}

### Initialize the model
Initialize the `Model` class with previous set params.

เลือก `Model` และตั้งค่า `parameters`

In [ ]:
from ibm_watson_machine_learning.foundation_models import Model

flan_ul2_model = Model(
    model_id=model_id_1, 
    params=parameters, 
    credentials=credentials,
    project_id=project_id)

flan_t5_model = Model(
    model_id=model_id_2,
    credentials=credentials,
    project_id=project_id)

In [ ]:
granite_model = Model(
    model_id=model_id_4,
    credentials=credentials,
    project_id=project_id)

In [ ]:
gptneox_model = Model(
    model_id=model_id_5,
    credentials=credentials,
    project_id=project_id)

In [ ]:
llama2_model = Model(
    model_id=model_id_3, 
    params=LLAMA_parameters, 
    credentials=credentials,
    project_id=project_id)

<a id="watsonxllm"></a>
## LangChain integration

`WatsonxLLM` is a wrapper around watsonx.ai models that provide chain integration around the models.

**Action:** For more details about `CustomLLM` check the [LangChain documentation](https://python.langchain.com/docs/modules/model_io/models/llms/custom_llm) 


`WatsonxLLM` คือ wrapper สำหรับ models ใน watsonx.ai ซึ่งจะทำให้สามารถใช้ model กับ Langchain ได้

**Action:** For more details about `CustomLLM` check the [LangChain documentation](https://python.langchain.com/docs/modules/model_io/models/llms/custom_llm) 


### Initialize the `WatsonxLLM` class.

In [ ]:
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

flan_ul2_llm = WatsonxLLM(model=flan_ul2_model)
flan_t5_llm = WatsonxLLM(model=flan_t5_model)

**Hint:** To use Chain interface from LangChain with watsonx.ai models you must call `model.to_langchain()` method. 

It returns `WatsonxLLM` wrapper compatible with LangChain CustomLLM specification.

**Hint:** ใช้ `model.to_langchain()` เพื่อทำให้โมเดลสามารถใช้ได้กับ LangChain


In [ ]:
flan_ul2_model.to_langchain()

You can print all set data about the WatsonxLLM object using the `dict()` method.

คุณสามารถพิมพ์ข้อมูลทั้งหมดเกี่ยวกับวัตสันเอลเอลเอ็็ม (WatsonxLLM) โดยใช้วิธี `dict()` 

In [ ]:
flan_ul2_llm.dict()

<a id="experiment"></a>
## Simple Sequential Chain experiment
The simplest type of sequential chain is called a `SimpleSequentialChain`, in which each step has a single input and output and the output of one step serves as the input for the following step.

The experiment will consist in generating a random question about any topic and answer the following question.

<a id="experiment"></a>
## การทดลอง Simple Sequential Chain

Chain ที่เรียงต่อกัน ชนิดที่ง่ายที่สุดคือ `SimpleSequentialChain` ในลักษณะนี้แต่ละขั้นตอนมีข้อมูลนำเข้าและข้อมูลส่งออกเพียงมิติเดียว ต่อเนื่องกัน และข้อมูลส่งออกของขั้นตอนหนึ่งนั้นใช้เป็นข้อมูลนำเข้าของขั้นตอนถัดไป

การทดลองจะประกอบด้วยการสร้างคำถามสุ่มเกี่ยวกับหัวข้อใดก็ได้และตอบคําถามต่อมา


An object called `PromptTemplate` assists in generating prompts using a combination of user input, additional non-static data, and a fixed template string.

In our case we would like to create two `PromptTemplate` objects which will be responsible for creating a random question and answering it.


`PromptTemplate` จะช่วยในการสร้าง Prompt โดยใช้การผสมระหว่างข้อมูลที่ผู้ใช้ป้อนเข้าไป ข้อมูลเพิ่มเติมที่ไม่ใช่ค่าคงที่, และเทมเพลตที่คงที่.

ในกรณีของเราเราต้องการสร้าง `PromptTemplate` สองเทมเพลตที่จะรับผิดชอบในการสร้างคำถามแบบสุ่มและการตอบคำถามนั้น.


In [ ]:
from langchain import PromptTemplate

prompt_1 = PromptTemplate(
    input_variables=["topic"], 
    template="Generate a random question about {topic}: Question: "
)
prompt_2 = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question: {question}",
)

We would like to add functionality around language models using `LLMChain` chain.

`prompt_to_flan_ul2` chain formats the prompt template whose task is to generate random question, passes the formatted string to LLM and returns the LLM output.


เราต้องการเพิ่มความสามารถเกี่ยวกับโมเดลโดยใช้ `LLMChain`.

 `prompt_to_flan_ul2` chain จะจัดรูปแบบข้อความ เทมเพลต ที่มีงานเป็นการสร้างคำถามแบบสุ่ม โดยรับ พรอมพ์ ที่จัดรูปแบบแล้วส่งไปที่ LLM และคืนผลลัพธ์จาก LLM ออกมา.

**Hint:** To use Chain interface from LangChain with watsonx.ai models you must call `model.to_langchain()` method. 

It returns `WatsonxLLM` wrapper compatible with LangChain CustomLLM specification.


**คำแนะนำ:** เพื่อใช้งานอินเทอร์เฟซ Chain จาก LangChain กับ Watsonx.ai คุณต้องเรียกใช้ `model.to_langchain()` ด้วย

เพื่อที่จะได้ `WatsonxLLM` wrapper ที่สามารถใช้งานกับ LangChain CustomLLM ได้


In [ ]:
from langchain.chains import LLMChain

prompt_to_flan_ul2 = LLMChain(llm=flan_ul2_model.to_langchain(), prompt=prompt_1)

`flan_to_t5` chain formats the prompt template whose task is to answer the question we got from `prompt_to_flan_ul2` chain, passes the formatted string to LLM and returns the LLM output.


`flan_to_t5` จัดรูปแบบเทมเพลตของพรอมป์ท์ที่มีหน้าที่ในการตอบคำถามที่ได้รับจาก `prompt_to_flan_ul2` จากนั้นส่งสตริงที่ถูกรูปแบบไปที่ LLM และคืนผลลัพธ์จาก LLM กลับมา.

In [ ]:
flan_to_t5 = LLMChain(llm=flan_t5_model.to_langchain(), prompt=prompt_2)

This is the overall chain where we run `prompt_to_flan_ul2` and `flan_to_t5` chains in sequence.

นี่คือโครงสร้างทั้งหมดที่เรารัน `prompt_to_flan_ul2` และ `flan_to_t5` ตามลำดับ.

In [ ]:
from langchain.chains import SimpleSequentialChain

qa = SimpleSequentialChain(chains=[prompt_to_flan_ul2, flan_to_t5], verbose=True)

Generate random question and answer to topic.

In [ ]:
topic = 'ชีวิต'
qa.run(topic)

# Thai

## Basic Chain of Thought as a Translator

เราสามารถ สร้างขั้นตอนการคิดสำหรับบอทได้ โดยเริ่มจากตีกรอบเฉพาะในหัวข้อที่เราสนใจ โดยการทำ chain of thought
ในกรณีนี้ เราจะใช้ ประโยค พร้อมคำแปล เกี่ยวกับการอาหารเป็นตัวอย่าง

จากตัวอย่างข้างล่าง จะสมามารถเห็นว่า การที่เราใช้ เทคนิค `chain of thought` สามาถทำให้โมเดลเรียนรู้ที่จะแปลความหมายในบริบทที่เกี่ยวข้องกับตัวอย่าง (ในที่นี้คืออาหาร) ได้

In [ ]:
question_template = '''
<s>[INST] <<SYS>>
INSTRUCTION:
คุณเป็นผู้สนใจความรู้ด้านอาหารฮาลาล ถามคำถามเป็นภาษาไทยในรูปแบบสั้น ๆ
You are person who interesting in Halal food knowledge. Ask the question in short and brief Thai language.
Produce the question using the steps as below.
Step 1: Generate the less than 10 words ENGLISH short question with simple grammar.
Step 2: Translate the ENGLISH short question into Thai language, use simple words.
AVOID the new line as much as possible. End the sentence after Step 2 is done.
Start your response with 'English QUESTION:'
<</SYS>>
INPUT:
TOPIC: {topic}
Step1: Generate English QUESTION:
Step2: Translate to Thai QUESTION:
[/INST]
'''


answer_template = '''
<s>[INST] <<SYS>>
INSTRUCTION:
คุณเป็นผู้เชี่ยวชาญด้านอาหารฮาลาล ตอบคำถามเป็นภาษาไทยในรูปแบบสั้น ๆ
You are expert in Halal food. Answer the question in short and brief Thai language.
Produce the answer using the steps as below.
Step 1: Generate the less than 5 words ENGLISH short answer with simple grammar.
Step 2: Translate the short ENGLISH answer into Thai language. You can mix English with Thai language if the words are technique or specific word.
AVOID the new line as much as possible. End the sentence after Step 2 is done.
Start your response with 'English ANSWER:'
<</SYS>>
{question}

Step 1: Generate English ANSWER:
Step 2: Translate to Thai ANSWER:
[/INST]
'''

In [ ]:
prompt_1 = PromptTemplate(
    input_variables=["topic"], 
    template=question_template
)

prompt_2 = PromptTemplate(
    input_variables=["question"],
    template=answer_template,
)


In [ ]:
LLAMA_parameters_q = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MAX_NEW_TOKENS: 80,
    GenParams.MIN_NEW_TOKENS: 1
}

LLAMA_parameters_a = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MAX_NEW_TOKENS: 70,
    GenParams.MIN_NEW_TOKENS: 1
}
llama2_chat_q = Model(
    model_id=model_id_3, 
    params=LLAMA_parameters_q, 
    credentials=credentials,
    project_id=project_id)

llama2_chat_a = Model(
    model_id=model_id_3, 
    params=LLAMA_parameters_a, 
    credentials=credentials,
    project_id=project_id)

In [ ]:
prompt_to_llama2_chat_q = LLMChain(llm=llama2_chat_q.to_langchain(), prompt=prompt_1)

In [ ]:
llama2_chat_q_to_llama2_chat_a = LLMChain(llm=llama2_chat_a.to_langchain(), prompt=prompt_2)

In [ ]:
qa = SimpleSequentialChain(chains=[prompt_to_llama2_chat_q, llama2_chat_q_to_llama2_chat_a], verbose=True)

In [ ]:
qa.run('ทอด')

<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to use Simple Squential Chain using custom llm WastonxLLM.
 
Check out our _[Online Documentation](https://ibm.github.io/watson-machine-learning-sdk/samples.html)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors: 
 **Mateusz Szewczyk**, Software Engineer at Watson Machine Learning.

Copyright © 2023 IBM. This notebook and its source code are released under the terms of the MIT License.